In [95]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity,linear_kernel

In [96]:
rec_system = pd.read_parquet("../datasets/rec_System.parquet").head(20000)
rec_system_copy = rec_system.copy()

In [97]:
rec_system_copy

,title,overview,name_genre,actors_names,director_names,tagline,vote_average
0,toy story,"Led by Woody, Andy's toys live happily in his ...","[Family, Comedy, Animation]","[Jim Varney, Tim Allen, Don Rickles, John Ratz...",[John Lasseter],None,7.7
1,jumanji,When siblings Judy and Peter discover an encha...,"[Fantasy, Family, Adventure]","[David Alan Grier, Kirsten Dunst, Lloyd Berry,...",[Joe Johnston],Roll the dice and unleash the excitement!,6.9
2,grumpier old men,A family wedding reignites the ancient feud be...,"[Comedy, Romance]","[Jack Lemmon, Ann-Margret, Sophia Loren, Daryl...",[Howard Deutch],Still Yelling. Still Fighting. Still Ready for...,6.5
3,waiting to exhale,"Cheated on, mistreated and stepped on, the wom...","[Drama, Comedy, Romance]","[Wesley Snipes, Whitney Houston, Dennis Haysbe...",[Forest Whitaker],Friends are the people who let you be yourself...,6.1
4,father of the bride part ii,Just when George Banks has recovered from his ...,[Comedy],"[George Newbern, Peter Michael Goetz, Kate McG...",[Charles Shyer],Just When His World Is Back To Normal... He's ...,5.7
...,...,...,...,...,...,...,...
19995,once upon a time in china and america,So goes to the U.S. to open a martial arts sch...,"[Western, Action]","[Chris Doughton, Roger Yuan, Johnny Koo, Fredd...","[Sammo Hung, Lau Kar-Wing]",None,5.7
19996,contract killer,"Financially troubled, a newbie hitman reluctan...","[Action, Comedy]","[Mitsuru Tokashiki, Gigi Leung, John Ching, Ke...",[Wei Tung],None,6.2
19997,the executor,Chow Yun-Fat and Danny Lee star in this police...,"[Drama, Action, Crime]","[Kwan Hoi-San, Danny Lee Sau-Yin, Lily Li Li-L...","[Chuan Chen, Chik Ngai-Hung, Yi-Hsiung Chi]",None,6.0
19998,never say... never!,"Paris, 1830. Valentin loses himself in alcohol...",[Comedy],"[Jean Dujardin, Lorella Cravotta, Jacques Herl...",[Eric Civanyan],None,4.8


In [98]:
rec_system.fillna({"overview":"[]",
                   "name_genre":"[]",
                   "actors_names":"[]",
                   "director_names":"[]",
                   "tagline":"[]",
                   "vote_average":rec_system["vote_average"].mean()},inplace=True)

In [99]:
rec_system["overview"].apply(lambda x: x.split())

0        [Led, by, Woody,, Andy's, toys, live, happily,...
1        [When, siblings, Judy, and, Peter, discover, a...
2        [A, family, wedding, reignites, the, ancient, ...
3        [Cheated, on,, mistreated, and, stepped, on,, ...
4        [Just, when, George, Banks, has, recovered, fr...
                               ...                        
19995    [So, goes, to, the, U.S., to, open, a, martial...
19996    [Financially, troubled,, a, newbie, hitman, re...
19997    [Chow, Yun-Fat, and, Danny, Lee, star, in, thi...
19998    [Paris,, 1830., Valentin, loses, himself, in, ...
19999    [Two, men,, fortyish,, worn, out, by, their, w...
Name: overview, Length: 20000, dtype: object

* Transformo la columna overview a una lista con la funcion `split`

In [100]:

rec_system["overview"] = rec_system["overview"].apply(lambda x: x.split())
rec_system["tagline"] = rec_system["tagline"].apply(lambda x: x.split())

In [101]:
def collapse(valor):
    valores =[]
    for i in valor:
       valores.append(i.replace(" ",""))
    return valores

In [102]:
rec_system["vote_average"].astype(str)

0        7.7
1        6.9
2        6.5
3        6.1
4        5.7
        ... 
19995    5.7
19996    6.2
19997    6.0
19998    4.8
19999    5.8
Name: vote_average, Length: 20000, dtype: object

In [103]:
rec_system["name_genre"]=rec_system["name_genre"].apply(collapse)
rec_system["actors_names"]=rec_system["actors_names"].apply(collapse)
rec_system["director_names"]=rec_system["director_names"].apply(collapse)
rec_system["tagline"]=rec_system["tagline"].apply(collapse)

In [104]:
rec_system["tags"] = rec_system["overview"] + rec_system["name_genre"] + rec_system["actors_names"] + rec_system["director_names"] + rec_system["tagline"] 

In [105]:
rec_system["tags"] = rec_system["tags"].apply(lambda x: "".join(x))

In [106]:
rec_system

,title,overview,name_genre,actors_names,director_names,tagline,vote_average,tags
0,toy story,"[Led, by, Woody,, Andy's, toys, live, happily,...","[Family, Comedy, Animation]","[JimVarney, TimAllen, DonRickles, JohnRatzenbe...",[JohnLasseter],[[]],7.7,"LedbyWoody,Andy'stoyslivehappilyinhisroomuntil..."
1,jumanji,"[When, siblings, Judy, and, Peter, discover, a...","[Fantasy, Family, Adventure]","[DavidAlanGrier, KirstenDunst, LloydBerry, Mal...",[JoeJohnston],"[Roll, the, dice, and, unleash, the, excitement!]",6.9,WhensiblingsJudyandPeterdiscoveranenchantedboa...
2,grumpier old men,"[A, family, wedding, reignites, the, ancient, ...","[Comedy, Romance]","[JackLemmon, Ann-Margret, SophiaLoren, DarylHa...",[HowardDeutch],"[Still, Yelling., Still, Fighting., Still, Rea...",6.5,Afamilyweddingreignitestheancientfeudbetweenne...
3,waiting to exhale,"[Cheated, on,, mistreated, and, stepped, on,, ...","[Drama, Comedy, Romance]","[WesleySnipes, WhitneyHouston, DennisHaysbert,...",[ForestWhitaker],"[Friends, are, the, people, who, let, you, be,...",6.1,"Cheatedon,mistreatedandsteppedon,thewomenareho..."
4,father of the bride part ii,"[Just, when, George, Banks, has, recovered, fr...",[Comedy],"[GeorgeNewbern, PeterMichaelGoetz, KateMcGrego...",[CharlesShyer],"[Just, When, His, World, Is, Back, To, Normal....",5.7,JustwhenGeorgeBankshasrecoveredfromhisdaughter...
...,...,...,...,...,...,...,...,...
19995,once upon a time in china and america,"[So, goes, to, the, U.S., to, open, a, martial...","[Western, Action]","[ChrisDoughton, RogerYuan, JohnnyKoo, FreddyJo...","[SammoHung, LauKar-Wing]",[[]],5.7,SogoestotheU.S.toopenamartialartsschool.Around...
19996,contract killer,"[Financially, troubled,, a, newbie, hitman, re...","[Action, Comedy]","[MitsuruTokashiki, GigiLeung, JohnChing, Keiji...",[WeiTung],[[]],6.2,"Financiallytroubled,anewbiehitmanreluctantlyta..."
19997,the executor,"[Chow, Yun-Fat, and, Danny, Lee, star, in, thi...","[Drama, Action, Crime]","[KwanHoi-San, DannyLeeSau-Yin, LilyLiLi-Li, Ch...","[ChuanChen, ChikNgai-Hung, Yi-HsiungChi]",[[]],6.0,ChowYun-FatandDannyLeestarinthispoliceactionfi...
19998,never say... never!,"[Paris,, 1830., Valentin, loses, himself, in, ...",[Comedy],"[JeanDujardin, LorellaCravotta, JacquesHerlin,...",[EricCivanyan],[[]],4.8,"Paris,1830.Valentinloseshimselfinalcohol,gambl..."


In [107]:
# max_features: maxima cantidad de palabras a vectorizar
# cv = CountVectorizer(max_features=5000, stop_words="english")
tv = TfidfVectorizer(max_features=5000, stop_words="english")

In [108]:
vector = tv.fit_transform(rec_system["tags"]).toarray()

In [109]:
tv.vocabulary_ # Chequeo palabras que se repitieron


{'andy': 501,
 'sheart': 3672,
 'alan': 266,
 'butshe': 986,
 'goodman': 1728,
 'bernie': 797,
 'swedding': 4096,
 'spregnant': 3952,
 'swife': 4110,
 'nina': 2868,
 'butthat': 989,
 'jr': 2347,
 'smith': 3863,
 'send': 3570,
 'thrilleractionadventurejean': 4543,
 'oneoftheworld': 2975,
 'alaska': 270,
 'allhero': 305,
 'anall': 377,
 'unfortunately': 4757,
 'sonny': 3917,
 'bones': 875,
 'butit': 977,
 'greed': 1738,
 'agetale': 220,
 'secrets': 3558,
 'yearslater': 4978,
 'smistress': 3862,
 'chip': 1121,
 'jacquesannaud': 2270,
 'shomosexuality': 3717,
 'year': 4972,
 'sdad': 3531,
 'roger': 3278,
 'clothes': 1189,
 'shakespeare': 3652,
 'eventually': 1502,
 'tosaveearth': 4643,
 'soon': 3919,
 'calif': 1022,
 'smind': 3859,
 'johnsmith': 2336,
 'spirited': 3943,
 'detective': 1324,
 'turned': 4722,
 'amanda': 333,
 'fiore': 1614,
 'butcan': 962,
 'finally': 1609,
 'afterlosingherjob': 196,
 'boss': 894,
 'rachel': 3205,
 '16': 11,
 'sfuneral': 3618,
 'marc': 2595,
 'olddaughter': 2

In [110]:
cosine_sim = linear_kernel(vector,vector)

In [111]:
indices = pd.Series(rec_system.index, index=rec_system["title"]).drop_duplicates()

In [112]:
# similarity = cosine_similarity(vector)

In [113]:
for i in rec_system[rec_system["title"]=="toy story"]["actors_names"]:
    print(i)

['JimVarney', 'TimAllen', 'DonRickles', 'JohnRatzenberger', 'ErikvonDetten', 'SarahFreeman', 'TomHanks', 'R.LeeErmey', 'LaurieMetcalf', 'PennJillette', 'WallaceShawn', 'JohnMorris', 'AnniePotts']


In [116]:
month_map={
    1: "Enero",
    2: "Febero"
}
month_map.

In [115]:
def recommend(movie,cosine_sim=cosine_sim):
    movie = movie.lower()
    index = indices[movie]
    sim_scores=list(enumerate(cosine_sim[index]))
    distances = sorted(sim_scores,reverse= True,key=lambda x: x[1])
    
    for i in distances[1:10]:
        # print(i[0])
        print(rec_system.iloc[i[0]].title)
recommend("toy stORy 2")

toy story 2
group sex
toy story
the first $20 million is always the hardest
at play in the fields of the lord
the gang's all here
life is sweet
mute witness
jumanji
